In [2]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf

import nltk
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

with open("intents.json", "r", encoding="utf-8") as f:
    intents = json.load(f)

words = []
classes = []
documents = []
ignoreLetters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words2.pkl', 'wb'))
pickle.dump(classes, open('classes2.pkl', 'wb'))

training = []
outputEmpty = [0] * len(classes)

for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

random.shuffle(training)
training = np.array(training)

trainX = training[:, :len(words)]
trainY = training[:, len(words):]

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(trainX, trainY, epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model2.h5')
print('Done')

Epoch 1/200
6/6 [==============================] - 0s 5ms/step - loss: 1.9354 - accuracy: 0.1111
Epoch 2/200
6/6 [==============================] - 0s 2ms/step - loss: 1.8657 - accuracy: 0.2222
Epoch 3/200
6/6 [==============================] - 0s 3ms/step - loss: 1.7963 - accuracy: 0.4074
Epoch 4/200
6/6 [==============================] - 0s 0s/step - loss: 1.7060 - accuracy: 0.3704
Epoch 5/200
6/6 [==============================] - 0s 427us/step - loss: 1.6598 - accuracy: 0.3333
Epoch 6/200
6/6 [==============================] - 0s 4ms/step - loss: 1.5342 - accuracy: 0.3704
Epoch 7/200
6/6 [==============================] - 0s 2ms/step - loss: 1.5089 - accuracy: 0.5556
Epoch 8/200
6/6 [==============================] - 0s 1ms/step - loss: 1.4897 - accuracy: 0.4815
Epoch 9/200
6/6 [==============================] - 0s 3ms/step - loss: 1.4640 - accuracy: 0.3333
Epoch 10/200
6/6 [==============================] - 0s 3ms/step - loss: 1.3899 - accuracy: 0.5185
Epoch 11/200
6/6 [==========